<h1 style="color:brown;">  Recurrent neural nets</h1> 

![Looping network](./img/RNN_colah.png)

##### RNNs can produce amazing results <a href ="http://karpathy.github.io/2015/05/21/rnn-effectiveness/">blog</a>

### Lesson plan 
1. Why classic neural nets are not enough?
2. Word embeddings - word2vec
3. Categorical embeddings
4. RNN 
5. Takeaways
6. Hands on word2vec

In [3]:
import numpy as np

### Classic nets vs. RNN's

Classic:
    - Inputs and outputs must be fixed-sized vectors
    - No idea of location or time 

RNNs: 

![](./img/diags.jpeg)

### Word embeddings

#### N-gram

-Which word produces the highest probability to be next given we have seen n specific other words before

In [ ]:
Words: Thank, you, Hello, goodbye

In [ ]:
If we have 4 words and we are looking at 2-gram? 
    Example: no. of times Thank you occurs divided by number of times Thank occurs

We need to calculate the probabilty of 
 - Thank Hello
 - Thank you
 - Thank goodbye
 - Thank Thank

So we needed to do 4 calculations

In [17]:
def how_many_calc_to_do(gram, voc_size):
    '''This function needs to calculate all combos 
    of words'''
    
    return np.prod(np.repeat(voc_size, gram))

In [15]:
how_many_calc_to_do(7, 10000)
# Notice that this is only an approxiamtion and it can be implemented in more efficient ways.

4477988020393345024

![](./img/one_hot_encoding_distance_on_3d.png)

#### Insight I: 
    we can actually just turn each word to a random vector sized 100/200/300, 
    train a classic neural net to predict the next word and update both the weights and the random vectors.
    You can think of it as just another layer of weights multiplying the one hot encoded inputs.
<a href="http://hunterheidenreich.com/blog/intro-to-word-embeddings/">word_embed blog</a>

<a href="https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa">word_embed blog II</a>


![W2V](./img/w2v.png)

##### Objective: maximize the sum of probabilities of each word given its observed window

This idea is very strong in comparison to other options: 

    Bag of words - just count occourences 
    TF-IDF - word is either informative or not but has no relation to other words
    one-hot encoder: for the computer paris-france is the same distance as paris-blabla

- Distance and direction are meaningful! King - man = Queen - woman
- Now the words massive and huge are similar!
- Extends to sentences, paragraphs and documents.

![](./img/see_attached_word_embed.png)

##### We have reduced the dimension of the vocublary by a big factor!
example: from 80,000 to 300

![](https://www.lemay.ai/demo/wordEmbedding/)

##### Insight I.I  The same thing can be applied to any categorical variable. 
##### With enough training data we can learn its continous position in space - state of the art

![categorical_embed](./img/categorical_embedding.png) # image I

![german_states](./img/german_states_mapped_2D.png) 

#### Insight II: 
        well, even if we can include many words (large n-gram), how can we capture context?
        If the text mentioned queen Mary and few pages later is talking about the queen, how will our network 
        know her name is Mary? 

### Idea I: Memory - your current choices are based on previous understanding

Add some cell in the network to keep previous memory and combine with current input to predict next word

![](./img/memory_rnn.png)

#### Problem: calculating the derivative (aka gradient) is problematic, either infinite or zero.

Imagine the memory at time t is the memory at time t-1 times a weight vector:
    $h_t = W*h_{t-1}$
Then:
    $h_t = W^t * h_0$ 
    
  $W > 1$ $h_t --> \infty$

### Solution: LSTM/GRU

<a href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/">LSTM/GRU blog</a>


![](./img/RNNs.png)

![](./img/LSTM_colah.png)

#### Idea II: gates: don't multiply, use addition for memory!

##### Components

    - cell state
    - candidates  

##### Gates
- forget - information to throw (0 means throw all from the cell state)
- input - what values we are going to update
- output - filter which values of the cell we are going to output 

The current cell state is the sum of forgetting and updating with new candidates

### Extension: attention

<a href="https://www.youtube.com/watch?v=SysgYptB198">Intuition</a>

######  - Translate part by part
###### -  Use attention weights - how much attention should you give to each word in the input (update weights to each new word)

![](./img/attention.png)

### Takeaways:
    

##### Word embeddings
- Word/categorical embeddings gives meaning to words in relation to one another
- Word/categorical embeddings are computationally efficient
- Training is done through a classic NN with small window around words

##### RNN
- Old generation RNNs suffered from exploding/vanishing gradients
- New generation RNNs (commonly LSTM or GRU) are using memory gates to mitigate this problem
- RNNs are just multiple copies of a NN connected by the hidden layer
- Training is done again by backpropogation
- Weights are shared accros all network
- RNN's can be used for any sequence. Unlike time series models can include both time and features.
- Are flexible in input and output sizes
- Amazing results in NLP, recommendations and many more.
- Many flavours - BRNN, CRNN...

##### Attention
- Typicall for translations/images
- Weight all the words in one language to decide how much they should influence input to translated language
- components: word weights, BRNN, RNN, context vectors.

### Hands on Word2Vec/word embedding

In [1]:
!pip install gensim

    100% |████████████████████████████████| 24.7MB 1.6MB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 1.1MB/s ta 0:00:011
    100% |████████████████████████████████| 133kB 924kB/s ta 0:00:01
    100% |████████████████████████████████| 5.6MB 1.4MB/s ta 0:00:011
  Stored in directory: /Users/basselhaidar/Library/Caches/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
Successfully built smart-open
awscli 1.16.195 has requirement botocore==1.12.185, but you'll have botocore 1.12.188 which is incompatible.
  Found existing installation: botocore 1.12.185
    Uninstalling botocore-1.12.185:
      Successfully uninstalled botocore-1.12.185


In [2]:
import gensim
import numpy as np
import json
import string

##### Reading in the data

In [4]:
with open('JEOPARDY_QUESTIONS1.json') as f:
    data = json.load(f)

In [5]:
len(data)

216930

In [6]:
# Let's look at the first element in our list
data[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [7]:
# Word2Vec requires that our text have the form of a list
# of 'sentences', where each sentence is itself a list of
# words. How can we put our _Jeopardy!_ clues in that shape?

text = []

for clue in data:
    sentence = clue['question'].translate(str.maketrans('', '',
                                                        string.punctuation)).split(' ')
    
    new_sent = []
    for word in sentence:
        new_sent.append(word.lower())
    
    text.append(new_sent)

In [8]:
# Let's check the new structure of our first clue
text[0]

['for',
 'the',
 'last',
 '8',
 'years',
 'of',
 'his',
 'life',
 'galileo',
 'was',
 'under',
 'house',
 'arrest',
 'for',
 'espousing',
 'this',
 'mans',
 'theory']

#####  Constructing the model

In [9]:
# simply a matter of
# instantiating a Word2Vec object.
model = gensim.models.Word2Vec(text, sg=1)
## sg means skip-gram

##### training 

In [10]:
# To train, call 'train()'!
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(11335282, 15849970)

In [11]:
# The '.wv' attribute stores the word vectors
model.wv

In [12]:
model.wv['child']

array([-0.13956743,  0.38852698, -0.1384079 ,  0.198164  ,  0.33369145,
        0.2581214 ,  0.11391034,  0.25246543,  0.4071368 , -0.30227414,
       -0.15263231, -0.02057954, -0.09294863,  0.00145738, -0.30328253,
        0.21440262,  0.00210786, -0.21836317,  0.01796227,  0.41315597,
        0.40303802,  0.07668992, -0.2985623 ,  0.2570073 , -0.07113824,
       -0.06148133,  0.0137752 ,  0.07562829,  0.08689208,  0.07090136,
       -0.35328564, -0.33349445,  0.40838748,  0.20534056,  0.141577  ,
       -0.01538388, -0.07834364,  0.4035147 , -0.22460756,  0.2623799 ,
        0.08859727,  0.21206166, -0.29069206, -0.54824275, -0.04540666,
       -0.47094166, -0.2649013 ,  0.4907699 , -0.44803396, -0.45546716,
       -0.36047983,  0.06202936, -0.08423393, -0.22419295, -0.38060832,
       -0.07530993, -0.1677465 , -0.7741386 ,  0.14172307,  0.1481089 ,
        0.68101007,  0.3418415 , -0.17733575, -0.5032891 , -0.7321344 ,
       -0.43678528, -0.06878028, -0.13061574, -0.12631771,  0.07

In [13]:
### model.wv methods
#### 'most_similar()' and 'similarity()'

In [14]:
model.wv.most_similar('happiness')

[('wherefore', 0.7530790567398071),
 ('shakespearebr', 0.7237969636917114),
 ('arise', 0.722869336605072),
 ('autumns', 0.7157022356987),
 ('ignorance', 0.710483193397522),
 ('loins', 0.7090731859207153),
 ('virtue', 0.708958625793457),
 ('hast', 0.7041985988616943),
 ('compassion', 0.7022799253463745),
 ('despair', 0.7014135122299194)]

In [15]:
model.wv.most_similar('furniture')

[('linen', 0.7072960138320923),
 ('ceramic', 0.6953456997871399),
 ('pottery', 0.6898662447929382),
 ('cabriole', 0.6842113137245178),
 ('flooring', 0.6841411590576172),
 ('chippendale', 0.683164119720459),
 ('jewelry', 0.6797876954078674),
 ('decorative', 0.6758967041969299),
 ('overalls', 0.6750195026397705),
 ('style', 0.6746268272399902)]

In [16]:
model.wv.similarity('furniture', 'jewelry')

0.67978764

In [17]:
model.wv.most_similar(positive=['cat', 'animal', 'pet', 'mammal'])

[('carnivore', 0.8021457195281982),
 ('arachnid', 0.7920539379119873),
 ('shorttailed', 0.7908763885498047),
 ('shorthaired', 0.784766674041748),
 ('wading', 0.7813202142715454),
 ('rodent', 0.7791703939437866),
 ('reptile', 0.775073230266571),
 ('seabird', 0.7748720645904541),
 ('scavenger', 0.7715741395950317),
 ('marsupial', 0.7680987119674683)]

In [18]:
model.wv.most_similar(positive=['cat', 'animal'], negative='pet')

[('camel', 0.3688696026802063),
 ('insect', 0.3652525544166565),
 ('lizard', 0.36179840564727783),
 ('rodent', 0.3524952530860901),
 ('breed', 0.33706122636795044),
 ('domesticated', 0.33276253938674927),
 ('arachnid', 0.3213215470314026),
 ('dog', 0.31988292932510376),
 ('extinction', 0.3150263726711273),
 ('monkey', 0.31444016098976135)]

In [19]:
model.wv.most_similar(positive=['king', 'woman'], negative='man', topn=3)

[('throne', 0.31459224224090576),
 ('empress', 0.3084597587585449),
 ('reign', 0.2718382477760315)]

In [20]:
model.wv.most_similar(positive='usa')

[('310', 0.6052267551422119),
 ('fargo', 0.6044845581054688),
 ('retitled', 0.5980973243713379),
 ('npr', 0.5924750566482544),
 ('pageant', 0.5898944139480591),
 ('coanchor', 0.588079571723938),
 ('tyra', 0.5847296714782715),
 ('reruns', 0.5786646604537964),
 ('ufos', 0.5750232934951782),
 ('xmas', 0.5724177360534668)]

In [21]:
model.wv.most_similar('canada')

[('commonwealth', 0.7211636304855347),
 ('zambia', 0.6969873309135437),
 ('marianas', 0.6916656494140625),
 ('turkmenistan', 0.6709186434745789),
 ('nigeria', 0.6619353294372559),
 ('48th', 0.6579661965370178),
 ('everglades', 0.6574159860610962),
 ('liechtenstein', 0.654984176158905),
 ('uruguay', 0.6526049375534058),
 ('albania', 0.6503781080245972)]

In [22]:
model.wv.most_similar('shakespeare')

[('shakespeares', 0.7567838430404663),
 ('sophocles', 0.7417217493057251),
 ('euripides', 0.6943586468696594),
 ('romeo', 0.6936426162719727),
 ('shakespearean', 0.6838273406028748),
 ('falstaff', 0.6737565994262695),
 ('ibsen', 0.6721268892288208),
 ('shaws', 0.6644858717918396),
 ('hamlet', 0.657096266746521),
 ('juliet', 0.6513713002204895)]

In [23]:
model.wv.most_similar(positive=['president', 'germany'], negative='usa')

[('emperors', 0.24580591917037964),
 ('dictator', 0.22600096464157104),
 ('ussr', 0.21086697280406952),
 ('presidents', 0.19787928462028503),
 ('nazi', 0.19467291235923767),
 ('regime', 0.19214406609535217),
 ('seized', 0.19173333048820496),
 ('russia', 0.18938156962394714),
 ('during', 0.1868162453174591),
 ('presidency', 0.18538056313991547)]

In [24]:
#### 'doesnt_match()'

In [25]:
model.wv.doesnt_match(['breakfast', 'lunch', 'frog', 'food'])

/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'frog'